# DEMO with Encoder-decoder

In [ ]:
from rnn_transformer_encoder_decoder import *

In [ ]:
labeled_output = [[],[]]
demo_dataset = TradeDGLDecoder({(1,2018):cluster_year[1,2018]},device,random_ratio = 1.0)

if lap_pos_enc:
    st = time.time()
    print("[!] Adding Laplacian positional encoding.")
    demo_dataset._add_laplacian_positional_encodings(net_params['pos_enc_dim'])
    print('Time LapPE:',time.time()-st)
    
if wl_pos_enc:
    st = time.time()
    print("[!] Adding WL positional encoding.")
    demo_dataset._add_wl_positional_encodings()
    print('Time WL PE:',time.time()-st)

demo_loader = DataLoader(demo_dataset, batch_size=batch_size, shuffle=False, collate_fn=test_dataset.collate)
_,_,_,_,labeled_output[0] = test_network(model, device, demo_loader)

In [ ]:
result_output_table = []

# add mask to simulate Australia/China cessation of trade

In [ ]:
import copy
cluster_year_mod = copy.deepcopy(cluster_year[1,2018])


cluster_year_mod.append({})
for j,k in cluster_year_mod[1]:
    if (j,k) not in cluster_year_mod[2]:
        cluster_year_mod[2][j,k] = cluster_year_mod[1][j,k]

#
j = le.transform([['Australia']])[0][0]
k = le.transform([['China']])[0][0]

jtar = le.transform([['Australia']])[0][0]
ktar = le.transform([['China']])[0][0]

for t in range(1,5):
    cluster_year_mod[2][j,k][0][t,0] = cluster_year_mod[1][j,k][0][t,0]
    cluster_year_mod[2][j,k][0][t,1] = cluster_year_mod[1][j,k][0][t,1]    
    cluster_year_mod[2][j,k][0][t,2] = cluster_year_mod[1][j,k][0][t,2]
    cluster_year_mod[2][j,k][0][t,3] = cluster_year_mod[1][j,k][0][t,3]
    cluster_year_mod[2][j,k][1][t] = 0.

demo_dataset = TradeDGLDecoder({(1,2018):cluster_year_mod},device,random_ratio = 1.0)


In [ ]:
if lap_pos_enc:
    st = time.time()
    print("[!] Adding Laplacian positional encoding.")
    demo_dataset._add_laplacian_positional_encodings(net_params['pos_enc_dim'])
    print('Time LapPE:',time.time()-st)
    
if wl_pos_enc:
    st = time.time()
    print("[!] Adding WL positional encoding.")
    demo_dataset._add_wl_positional_encodings()
    print('Time WL PE:',time.time()-st)

demo_loader = DataLoader(demo_dataset, batch_size=batch_size, shuffle=False, collate_fn=test_dataset.collate)
_,_,_,_,labeled_output[1] = test_network(model, device, demo_loader)

In [ ]:
import matplotlib.pyplot as plt
import networkx as nx

fig, axs = plt.subplots(5,2,figsize=(15, 35))


for n in range(2):
    k = 0
    for t, vals in labeled_output[n]:
        axs[t,n].set_title(t + 2018)
        G = nx.Graph()
        
        for i,j,v in vals:
            i_ = demo_dataset.le.inverse_transform([[int(i)]])[0]
            j_ = demo_dataset.le.inverse_transform([[int(j)]])[0]
            G.add_edge(le.inverse_transform([[i_]])[0][0], le.inverse_transform([[j_]])[0][0], weight= float(v))
            if n == 0:
                result_output_table.append([t + 2018,le.inverse_transform([[i_]])[0][0], le.inverse_transform([[j_]])[0][0], float(v)* features_max[0]])                        
            else:
                result_output_table[k] += [float(v)* features_max[0]]  
            k += 1    
        
        elarge = [(u, v) for (u, v, d) in G.edges(data=True) if d["weight"] > 0.2]
        ealarge = [(u, v) for (u, v, d) in G.edges(data=True) if ((d["weight"] > 0.15) and (d["weight"] <= 0.2))]
        emed = [(u, v) for (u, v, d) in G.edges(data=True) if ((d["weight"] > 0.05) and (d["weight"] <= 0.15))]
        esmall = [(u, v) for (u, v, d) in G.edges(data=True) if d["weight"] <= 0.05]
    
    
        if len(elarge + esmall+emed+ealarge) > 0:
            layout = nx.spring_layout(G, seed=7,weight=None)
            #nx.draw(G, layout)
            nx.draw_networkx_nodes(G, layout, node_size=200,alpha=0.4,ax=axs[t,n])
        
            # edges
            nx.draw_networkx_edges(G, layout, edgelist=elarge, width=1.5,alpha=0.5,edge_color="r",ax=axs[t,n])
            nx.draw_networkx_edges(
                G, layout, edgelist=ealarge, width=1.0, alpha=0.7, edge_color="y",ax=axs[t,n]
            )           
            nx.draw_networkx_edges(
                G, layout, edgelist=emed, width=0.9, alpha=0.7, edge_color="g",ax=axs[t,n]
            )        
            nx.draw_networkx_edges(
                G, layout, edgelist=esmall, width=0.5, alpha=0.7, edge_color="b",ax=axs[t,n]
            )
            
            # node labels
            nx.draw_networkx_labels(G, layout, font_size=12, font_family="sans-serif",ax=axs[t,n])
            # edge weight labels
            #edge_labels = nx.get_edge_attributes(G, "weight")
            #nx.draw_networkx_edge_labels(G, layout, edge_labels,font_size=6)
plt.tight_layout()
plt.savefig("constrained_model.png")        
plt.show()       

In [ ]:
pd.set_option('display.max_rows', 500)

pd.DataFrame(result_output_table,columns=["Year","Exporter","Importer","Unconstrained","Constrained: Au->China"]).to_csv("trade_model.csv",sep=";")

# add mask to simulate Australia/Indonesia cessation of trade

In [ ]:
demo_dataset = TradeDGLDecoder({(1,2018):cluster_year[1,2018]},device,random_ratio = 1.0)

if lap_pos_enc:
    st = time.time()
    print("[!] Adding Laplacian positional encoding.")
    demo_dataset._add_laplacian_positional_encodings(net_params['pos_enc_dim'])
    print('Time LapPE:',time.time()-st)
    
if wl_pos_enc:
    st = time.time()
    print("[!] Adding WL positional encoding.")
    demo_dataset._add_wl_positional_encodings()
    print('Time WL PE:',time.time()-st)

demo_loader = DataLoader(demo_dataset, batch_size=batch_size, shuffle=False, collate_fn=test_dataset.collate)
_,_,_,_,labeled_output[0] = test_network(model, device, demo_loader)

In [ ]:
import copy
cluster_year_mod = copy.deepcopy(cluster_year[1,2018])


cluster_year_mod.append({})
for j,k in cluster_year_mod[1]:
    if (j,k) not in cluster_year_mod[2]:
        cluster_year_mod[2][j,k] = cluster_year_mod[1][j,k]

#
j = le.transform([['Australia']])[0][0]
k = le.transform([['Viet Nam']])[0][0]

jtar = le.transform([['Australia']])[0][0]
ktar = le.transform([['Viet Nam']])[0][0]

for t in range(1,5):
    cluster_year_mod[2][j,k][0][t,0] = cluster_year_mod[1][j,k][0][t,0]
    cluster_year_mod[2][j,k][0][t,1] = cluster_year_mod[1][j,k][0][t,1]    
    cluster_year_mod[2][j,k][0][t,2] = cluster_year_mod[1][j,k][0][t,2]
    cluster_year_mod[2][j,k][0][t,3] = cluster_year_mod[1][j,k][0][t,3]
    cluster_year_mod[2][j,k][1][t] = 0.

demo_dataset = TradeDGLDecoder({(1,2018):cluster_year_mod},device,random_ratio = 1.0)



In [ ]:
if lap_pos_enc:
    st = time.time()
    print("[!] Adding Laplacian positional encoding.")
    demo_dataset._add_laplacian_positional_encodings(net_params['pos_enc_dim'])
    print('Time LapPE:',time.time()-st)
    
if wl_pos_enc:
    st = time.time()
    print("[!] Adding WL positional encoding.")
    demo_dataset._add_wl_positional_encodings()
    print('Time WL PE:',time.time()-st)

demo_loader = DataLoader(demo_dataset, batch_size=batch_size, shuffle=False, collate_fn=test_dataset.collate)
_,_,_,_,labeled_output[1] = test_network(model, device, demo_loader)

In [ ]:
import matplotlib.pyplot as plt
import networkx as nx

fig, axs = plt.subplots(5,2,figsize=(15, 35))


for n in range(2):
    k = 0
    for t, vals in labeled_output[n]:
        axs[t,n].set_title(t + 2018)
        G = nx.Graph()
        
        for i,j,v in vals:
            i_ = demo_dataset.le.inverse_transform([[int(i)]])[0]
            j_ = demo_dataset.le.inverse_transform([[int(j)]])[0]
            G.add_edge(le.inverse_transform([[i_]])[0][0], le.inverse_transform([[j_]])[0][0], weight= float(v))
            if n == 0:
                result_output_table.append([t + 2018,le.inverse_transform([[i_]])[0][0], le.inverse_transform([[j_]])[0][0], float(v)* features_max[0]])                        
            else:
                result_output_table[k] += [float(v)* features_max[0]]  
            k += 1    
        
        elarge = [(u, v) for (u, v, d) in G.edges(data=True) if d["weight"] > 0.2]
        ealarge = [(u, v) for (u, v, d) in G.edges(data=True) if ((d["weight"] > 0.15) and (d["weight"] <= 0.2))]
        emed = [(u, v) for (u, v, d) in G.edges(data=True) if ((d["weight"] > 0.05) and (d["weight"] <= 0.15))]
        esmall = [(u, v) for (u, v, d) in G.edges(data=True) if d["weight"] <= 0.05]
    
    
        if len(elarge + esmall+emed+ealarge) > 0:
            layout = nx.spring_layout(G, seed=7,weight=None)
            #nx.draw(G, layout)
            nx.draw_networkx_nodes(G, layout, node_size=200,alpha=0.4,ax=axs[t,n])
        
            # edges
            nx.draw_networkx_edges(G, layout, edgelist=elarge, width=1.5,alpha=0.5,edge_color="r",ax=axs[t,n])
            nx.draw_networkx_edges(
                G, layout, edgelist=ealarge, width=1.0, alpha=0.7, edge_color="y",ax=axs[t,n]
            )           
            nx.draw_networkx_edges(
                G, layout, edgelist=emed, width=0.9, alpha=0.7, edge_color="g",ax=axs[t,n]
            )        
            nx.draw_networkx_edges(
                G, layout, edgelist=esmall, width=0.5, alpha=0.7, edge_color="b",ax=axs[t,n]
            )
            
            # node labels
            nx.draw_networkx_labels(G, layout, font_size=12, font_family="sans-serif",ax=axs[t,n])
            # edge weight labels
            #edge_labels = nx.get_edge_attributes(G, "weight")
            #nx.draw_networkx_edge_labels(G, layout, edge_labels,font_size=6)
plt.tight_layout()
plt.savefig("constrained_model2.png")        
plt.show()       

In [ ]:
pd.set_option('display.max_rows', 500)

pd.DataFrame(result_output_table,columns=["Year","Exporter","Importer","Unconstrained","Constrained: Au->Ind"]).to_csv("trade_model2.csv",sep=";")